In [3]:
import wandb
import os
os.environ["WANDB_SILENT"] = "true"

import numpy as np
import pandas as pd

import sys
sys.path.insert(0, "../..")

from helpers import load_variational_mgt_model
from model import GrooveTransformerEncoderVAE
import torch

from umap import UMAP

from bokeh.palettes import inferno, Category20b
from bokeh.core.enums import MarkerType
from bokeh.plotting import figure, show, save
from bokeh.io import output_notebook
output_notebook()




Holoviews not installed, will not be able to generate violin plots


Loading BokehJS ...

In [35]:
run = wandb.init()
links = {
    "GOOD_AVERAGE_glamorous-sweep-62": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v283",
    "GOOD_azure-sweep-54": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v279",
    "GOOD_apricot-sweep-17": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v242",
    "GOOD_hearty-sweep-60": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v280",
    "GOOD_worldly-sweep-22": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v245",
    "GOOD_legendary-sweep-5": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v230",
    "drawn_river_6": "mmil_vae_g2d/beta_annealing_study/model_epoch_100:v2",
    "worldly-firebrand-5": "mmil_vae_g2d/beta_annealing_study/model_epoch_100:v1",
    "noble-field-7": "mmil_vae_g2d/beta_annealing_study/model_epoch_100:v3",
    "young-violet-12": "mmil_vae_g2d/beta_annealing_study/model_epoch_700:v0",
    "kind-gorge-14": "mmil_vae_g2d/beta_annealing_study/model_epoch_500:v1",
    "confused-feather-17": "mmil_vae_g2d/beta_annealing_study/model_epoch_090:v0"
    
}
run_name = "confused-feather-17"
artifact = run.use_artifact(links[run_name], type='model')
artifact_dir = artifact.download()

In [45]:
from data import load_gmd_hvo_sequences

test_set = load_gmd_hvo_sequences(
    dataset_setting_json_path = "../../data/dataset_json_settings/4_4_Beats_gmd.json", 
    subset_tag = "test", 
    force_regenerate=False)

INFO:data.Base.dataLoaders:Loading gmd dataset
INFO:data.Base.dataLoaders:Loading Cached Version from: data/gmd/resources/cached/beat_division_factor_[4]/drum_mapping_label_ROLAND_REDUCED_MAPPING/beat_type_['beat']_time_signature_['4-4']


In [46]:
model = load_variational_mgt_model(os.path.join(artifact_dir, "090.pth"))

Offset activation is sigmoid, bias is initialized to 0.5


In [47]:
gt_sample = test_set[0]
groove = torch.tensor([gt_sample.flatten_voices(reduce_dim=True)], dtype=torch.float32)
gt_sample.metadata

{'Source': 'Groove MIDI Dataset',
 'drummer': 'drummer1',
 'session': 'eval_session',
 'loop_id': 'drummer1/eval_session/10:000',
 'master_id': 'drummer1/eval_session/10',
 'style_primary': 'soul',
 'style_secondary': 'groove10',
 'bpm': '102',
 'beat_type': 'beat',
 'time_signature': '4-4',
 'full_midi_filename': 'drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid',
 'full_audio_filename': 'drummer1/eval_session/10_soul-groove10_102_beat_4-4.wav'}

In [48]:
mu, logvar = model.encode_to_mu_logvar(groove)
latent_z = model.reparametrize(mu, logvar)
latent_z

tensor([[-0.6900, -0.4444,  0.3034,  0.1347,  0.9214,  0.2585, -0.6418, -1.0061,
          0.8347, -1.5086,  1.0133,  3.6551, -0.1974,  0.4088,  0.8435,  0.3896,
         -0.2774,  0.0580, -0.6307,  0.4291, -0.3979,  1.8178, -2.7101,  0.0621,
         -0.0853, -1.5414,  0.1522, -1.5367, -1.8749,  2.0110, -1.8613,  1.2111,
          0.3769,  1.3507,  0.2016, -0.5687,  0.2932,  0.8866, -0.7119,  0.8716,
         -1.4853,  0.1980, -0.0418, -2.5995, -1.1396, -0.6304,  2.1820,  0.1831,
          0.1847,  0.0907,  0.2572, -0.1507, -1.9242, -0.5474,  0.1977,  0.2870,
          0.5377, -1.3959, -0.8147, -1.7201, -0.0761, -0.2512, -0.5325, -0.6322,
          0.5361, -1.8775, -0.5158,  0.6766, -0.1270,  1.5611,  0.3009,  0.7009,
         -0.0947,  1.3176, -0.3054, -0.4037, -1.3737,  1.0943,  0.8008,  0.4314,
          0.3708, -1.1106, -0.0508, -0.1025, -0.9768,  0.1179,  0.8844,  0.9830,
          0.8999,  0.2081,  1.7322, -0.9414, -0.5995,  0.0268,  0.7533, -1.2559,
          0.4019, -1.1851, -

In [49]:
voice_thresholds = [0.5] * 9
voice_max_count_allowed = [32] * 9
h, v, o = model.sample(latent_z=latent_z,
                       voice_thresholds=voice_thresholds,
                       voice_max_count_allowed=voice_max_count_allowed,
                       return_concatenated=False,
                       sampling_mode=0)
print(h, v, o)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0., 0.],
         [0.,

In [50]:
latents = []
labels = []
metadatas = []
use_all_styles = True

for gt_sample in test_set:
    if (use_all_styles or (gt_sample.metadata["style_primary"] in ["rock", "funk", "afrobeat"])):
        metadatas.append(gt_sample.metadata)
        labels.append(gt_sample.metadata["style_primary"])
        groove = torch.zeros((1, 32, 3))
        flattened_ = torch.tensor([gt_sample.flatten_voices(reduce_dim=True)], dtype=torch.float32)[:,:32, :]
        t_steps = flattened_.shape[1]
        groove[:, :t_steps, :] = flattened_
        mu, logvar = model.encode_to_mu_logvar(groove)
        latent_z = model.reparametrize(mu, logvar)
        latents.append(latent_z.detach().cpu().numpy())

latents = np.array(latents).squeeze(1)
features = np.expand_dims(latents, -1) # we use each dimension of latent_z as a feature
feature_labels = [f"z_{dim}" for dim in range(features.shape[1])]

In [51]:
data = {"style_primary": [style for style in labels]}
data.update({f"z_{dim_i}": latents[:, dim_i] for dim_i in range(features.shape[1])})
df = pd.DataFrame(data)
df.head()

,style_primary,z_0,z_1,z_2,z_3,z_4,z_5,z_6,z_7,z_8,...,z_118,z_119,z_120,z_121,z_122,z_123,z_124,z_125,z_126,z_127
0,soul,0.979023,0.233069,0.111509,-0.504703,2.631382,1.752098,2.560483,1.410372,1.800011,...,-0.808512,0.571675,-0.014047,-0.317053,-0.762000,1.281474,0.140603,0.006950,0.514216,0.344799
1,soul,-2.879863,-1.257560,-0.185040,0.118882,1.159302,0.743724,-0.208088,-0.228568,-2.161211,...,0.499796,-0.505619,0.637984,-2.138556,0.382957,-0.639571,0.179651,-1.035419,-0.065432,-1.437978
2,soul,-1.170722,-1.886661,-0.449983,-0.663444,1.399997,-0.290256,0.301659,0.001132,-1.826960,...,1.817958,0.925579,0.261184,-0.513301,-1.698608,0.529939,-0.148180,-0.787439,-2.422021,0.403708
3,soul,-2.157297,-0.378227,0.249099,-0.392280,-0.131119,0.760492,-0.543589,-0.886329,-0.038678,...,1.672265,-0.934707,1.234030,0.396867,-0.783238,-0.612750,1.432293,1.400588,1.634989,-0.319299
4,soul,-1.505247,0.859131,-0.842252,0.223342,-0.079079,0.245053,-0.249977,0.785670,0.132599,...,-1.096236,-0.075319,-1.715709,-0.346729,-1.290694,-0.274146,-0.261431,-0.168129,1.380819,-1.254665


In [52]:
use_z_dims = np.arange(32, 64, 1)
umap_df_keys = [f"z_{ix}" for ix in use_z_dims]
umap_df = df
for key in df.keys():
    umap_df = umap_df if key in umap_df_keys else umap_df.drop(key, axis=1)


In [53]:
embedding_dims = 2
umap = UMAP(n_components=embedding_dims, metric="cosine", n_neighbors=20)
embeddings = umap.fit_transform(umap_df)

In [54]:



styles = sorted(df.style_primary.unique())
hues = inferno(len(styles))
hues = Category20b[20]
style_hue_map = {f"{style}": hues[i] for i, style in enumerate(styles)}



marker_map = {style: bases[ix] for ix, style in enumerate(styles)}

In [55]:
embedding_dict = dict()
styles = df.style_primary


for ix, style in enumerate(styles):
    if(not style in embedding_dict):
        embedding_dict.update({style: ([], [])})
    embedding_dict[style][0].append(embeddings[ix][0])
    embedding_dict[style][1].append(embeddings[ix][1])
    

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)

for key, values in embedding_dict.items():
    p.scatter(values[0], values[1],
              fill_color=style_hue_map[key], 
              line_color=None, legend_label=key, marker=marker_map[key], size=12)
p.legend.click_policy="hide"
show(p)

save(p, f"UMAP_{run_name}.html")

embedding_dict

In [56]:
# from sklearn.manifold import TSNE
# umap = TSNE(n_components=embedding_dims)
# embeddings = umap.fit_transform(df.drop("style_primary", axis=1))

In [57]:
# import numpy as np

# from bokeh.plotting import figure, show, save
# from bokeh.io import output_notebook

# TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

# p = figure(tools=TOOLS)

# p.scatter(embeddings[:, 0], embeddings[:, 1],
         #  fill_color=colors, fill_alpha=0.6,
          # line_color=None)

# save(p, "TSNE.html") 